In [19]:
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller


In [20]:
df=pd.read_csv("gibf1.csv")


In [21]:

def find_stationary_intervals(df, column_name='closePrice', window_size=20, p_value_threshold=0.001):
    intervals = []
    series = df[column_name]  
    
    for i in range(len(series) - window_size + 1):
        window = series[i:i + window_size]
        adf_result = adfuller(window, autolag="AIC")
        p_value = adf_result[1]
        if p_value < p_value_threshold:
            intervals.append((i, i + window_size))  
            
    return intervals


stationary_intervals = find_stationary_intervals(df, column_name='closePrice', window_size=20)

print("Stationary intervals (start, end):", stationary_intervals)


stable_sections = [df.iloc[start:end] for start, end in stationary_intervals]


print("First stable section:\n", stable_sections[0])


Stationary intervals (start, end): [(1, 21), (3, 23), (10, 30), (12, 32), (13, 33), (17, 37), (18, 38), (20, 40), (21, 41), (26, 46), (31, 51), (32, 52), (37, 57), (40, 60), (43, 63), (48, 68), (53, 73), (54, 74), (58, 78), (59, 79), (62, 82), (64, 84), (65, 85), (66, 86), (69, 89), (71, 91), (74, 94), (99, 119), (103, 123), (107, 127), (113, 133), (117, 137), (124, 144), (128, 148), (140, 160), (142, 162), (144, 164), (158, 178), (161, 181), (162, 182), (193, 213), (194, 214), (202, 222), (205, 225)]
First stable section:
    businessDate  openPrice  highPrice  lowPrice  previousDayClosePrice  \
1    2024-02-08       8.41       8.59      8.41                   8.42   
2    2024-02-11       8.45       8.45      8.45                   8.59   
3    2024-02-12       8.30       8.45      8.30                   8.45   
4    2024-02-13       8.30       8.46      8.30                   8.45   
5    2024-02-14       8.33       8.79      8.33                   8.46   
6    2024-02-15       8.84

In [22]:
stable_sections_df = []

In [23]:
for idx, stable_section in enumerate(stable_sections):
    stable_section['Stable_Section'] = idx + 1  
    stable_sections_df.append(stable_section)

C:\Users\shara\AppData\Local\Temp\ipykernel_4876\3040077910.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stable_section['Stable_Section'] = idx + 1
C:\Users\shara\AppData\Local\Temp\ipykernel_4876\3040077910.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stable_section['Stable_Section'] = idx + 1
C:\Users\shara\AppData\Local\Temp\ipykernel_4876\3040077910.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = va

In [24]:
stable_sections_combined = pd.concat(stable_sections_df)

In [25]:
stable_sections_combined.reset_index(drop=True, inplace=True)

In [26]:
print(stable_sections_combined.head())

  businessDate  openPrice  highPrice  lowPrice  previousDayClosePrice  \
0   2024-02-08       8.41       8.59      8.41                   8.42   
1   2024-02-11       8.45       8.45      8.45                   8.59   
2   2024-02-12       8.30       8.45      8.30                   8.45   
3   2024-02-13       8.30       8.46      8.30                   8.45   
4   2024-02-14       8.33       8.79      8.33                   8.46   

   fiftyTwoWeekHigh  lastTradedPrice  totalTradedQuantity  closePrice  \
0               9.7             8.59                 1700        8.59   
1               9.7             8.45                 1400        8.45   
2               9.7             8.45                 4400        8.45   
3               9.7             8.46                 5200        8.46   
4               9.7             8.67                 4380        8.67   

        EMA      MACD    Signal  Stable_Section  
0  8.446154  0.013561  0.002712               1  
1  8.446746  0.012864 

In [27]:
stable_sections_combined.to_csv('gibf1_stable_sections.csv', index=False)